## Notes/Ideas
In this Python notebook I will keep track of various things, including notes/documentation on how the old dBASE program works as well as explorations of ideas for the new software that I can play around with here before putting them in the real program.

In [244]:
# Libraries/Modules
import pandas as pd
import numpy as np
import os
import customtkinter as ctk
import functions as fn
from datetime import datetime

# Static variables
data_dir = 'C:\\STMNU2\\data'
dbf_dir = 'C:\\dbase\\gymtek'

### DBF Files

In [42]:
# Get a list of all the .dbf files that exist in the old program directory
dbf_files = [file for file in os.listdir(dbf_dir) if file.lower().endswith('.dbf')]
print(dbf_files)
# Need to ask Dad which of these are important. For now go ahead and convert everything to .csv
# so we can easily pull them into a Pandas dataframe and view them
for file in dbf_files:
    fn.dbf_to_csv(file)

['14toNov.dbf', 'BILLING.DBF', 'clsbymon.dbf', 'CLSBYMOX.DBF', 'clsbymxx.dbf', 'Copy (2) of clsbymon.dbf', 'Copy (2) of CLSBYMOX.DBF', 'Copy (2) of MNCHECK.DBF', 'Copy of clsbymon.dbf', 'Copy of CLSBYMOX.DBF', 'Copy of FIXMOX.DBF', 'Copy of MNCHECK.DBF', 'Copy of STUD00.dbf', 'Copy of STUD99.dbf', 'FINDBEN.dbf', 'MNCHECK.DBF', 'MNCHECK1.DBF', 'MONNEW.DBF', 'MONNEWS.DBF', 'NEWMARCH.DBF', 'STUD00 (1).dbf', 'STUD00.DBF', 'STUD00BA.DBF', 'STUD00BACK.DBF', 'STUD00withSarah.DBF', 'STUD1999.DBF', 'STUD2001.DBF', 'STUD2002.DBF', 'STUD2002B.DBF', 'STUD2002from2004.DBF', 'STUD2003.DBF', 'STUD2004.dbf', 'STUD2005.DBF', 'STUD2006.dbf', 'STUD2007.dbf', 'STUD2008.dbf', 'STUD2009.dbf', 'STUD2010.dbf', 'STUD2011.dbf', 'STUD2012.DBF', 'STUD2013.dbf', 'STUD2014.dbf', 'STUD2015.dbf', 'STUD2016.dbf', 'STUD2017.dbf', 'STUD2018.DBF', 'STUD2019 (2).dbf', 'STUD2019.dbf', 'STUD2020.dbf', 'STUD2021.dbf', 'STUD2022.DBF', 'STUD6MO.DBF', 'STUD99.DBF', 'STUD99.DBF.dbf', 'STUD99back.dbf', 'STUD99O.dbf', 'STUD99ol.db

#### F1: Student Info
Replicating the functionality of the 'student info' feature in the original program (press F1 on main menu)

In [52]:
# Find out which 'STUD' dbf has the most records
csv_files = [file for file in os.listdir(data_dir) if 'stud' in file.lower()]
record_counts = []
for file in csv_files:
    record_counts.append(pd.read_csv(os.path.join(data_dir, file)).shape[0])

most_records = csv_files[np.array(record_counts).argmax()]
print(most_records)
students = pd.read_csv(os.path.join(data_dir, most_records))
display(students)

ValueError: attempt to get argmax of an empty sequence

In [25]:
# 'lname' will represent the user input, case-insensitive
lname = ctk.CTkInputDialog(text="Last Name:", title="Student Info").get_input().upper()

# Find all students with a match, and then select the first record alphabetically 
first_match = students[students['LNAME'].str.upper().str.startswith(lname, na=False)
                ].sort_values(by=['LNAME','FNAME']
                ).head(1)
pd.set_option('display.max_columns', None)
display(first_match)


,FNAME,MIDDLE,LNAME,SEX,BIRTHDAY,CLASS,LEVEL,ADDRESS,CITY,STATE,ZIP,PHONE,NOTE1,NOTE2,NOTE3,INSTRUCTOR,DAYTIME,INST2,DAYTIME2,INST3,DAYTIME3,REGFEE,REGFEEDATE,EMAIL,STATUS,ENROLLDATE,STUDENTNO,MOMNAME,DADNAME,MONTHLYFEE,BALANCE,JANPAY,JANDATE,JANBILL,FEBPAY,FEBDATE,FEBBILL,MARPAY,MARDATE,MARBILL,APRPAY,APRDATE,APRBILL,MAYPAY,MAYDATE,MAYBILL,JUNPAY,JUNDATE,JUNBILL,JULPAY,JULDATE,JULBILL,AUGPAY,AUGDATE,AUGBILL,SEPPAY,SEPDATE,SEPBILL,OCTPAY,OCTDATE,OCTBILL,NOVPAY,NOVDATE,NOVBILL,DECPAY,DECDATE,DECBILL,REGBILL
790,ELIZABETH,NaN,ZUCKER,NaN,1998-07-29,GB,0,PO BOX 3537,PLANT CITY,FL,33563,H/813-659-2605 DC/813-763-5853,TAKING SUMMER09 OFF AB,LM 8/24 TO SEE IF COMING,BACK KO,AMANDA,M 5:05,NaN,NaN,NaN,NaN,21.0,2008-11-04,NaN,False,2008-04-15,1791,MARIA C/813-624-5241,STEVEN,75.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN


In [69]:
student_info = students.iloc[1110]
student_info = student_info.astype('string').str.title()
student_info['MONTHLYFEE']

'65.0'

### New Database Structure
At the moment, the important tables are `STUD00.dbf` and `clsbymon.dbf`. The first table contains student information since the start of Gymtek, and for each student, their entire payment history *for the current year only*. The second table contains class information, including every student enrolled in that class. As such these files have a lot of fields (in `STUD00.dbf` each month gets several columns, in `clsbymon.dbf` there's a ton of placeholder columns for students). 

I am going to use what I've learned in my work experience to reorganize the database structure into something that more closely resembles a relational database. For example, a table `students.csv` that contains just a few columns regarding a student's personal information, a `classes.csv` table, `family.csv`, `payment.csv`, etc. that will keep different information organized and can easily be linked using keys from each table (i.e. `STUDENT_ID`, `CLASS_ID`, etc.). The code cells below are for figuring out the best way to import/transform the data into this new structure, and once polished I will put the code into `functions.py`.

Dad wants to be able to see the records in the same format as the current .dbf files, but this format will make it difficult to write code for this software in the way I envision it, so I am also going to maintain/update files in this old style alongside the new ones. The program will not use those directly but they will always be up-to-date with the new database, so that they can be cross-referenced when needed.

In [33]:
# Load students file and list all fields
STUD00 = pd.read_csv('Data\\dbf_format\\STUD00.csv').dropna(subset=['FNAME','LNAME']).reset_index(drop=True)
STUD00.insert(0, 'STUDENT_ID', STUD00.index + 1)
print(STUD00.shape)
print(STUD00.columns)

(5923, 69)
Index(['STUDENT_ID', 'FNAME', 'MIDDLE', 'LNAME', 'SEX', 'BIRTHDAY', 'CLASS',
       'LEVEL', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'PHONE', 'NOTE1', 'NOTE2',
       'NOTE3', 'INSTRUCTOR', 'DAYTIME', 'INST2', 'DAYTIME2', 'INST3',
       'DAYTIME3', 'REGFEE', 'REGFEEDATE', 'EMAIL', 'STATUS', 'ENROLLDATE',
       'STUDENTNO', 'MOMNAME', 'DADNAME', 'MONTHLYFEE', 'BALANCE', 'JANPAY',
       'JANDATE', 'JANBILL', 'FEBPAY', 'FEBDATE', 'FEBBILL', 'MARPAY',
       'MARDATE', 'MARBILL', 'APRPAY', 'APRDATE', 'APRBILL', 'MAYPAY',
       'MAYDATE', 'MAYBILL', 'JUNPAY', 'JUNDATE', 'JUNBILL', 'JULPAY',
       'JULDATE', 'JULBILL', 'AUGPAY', 'AUGDATE', 'AUGBILL', 'SEPPAY',
       'SEPDATE', 'SEPBILL', 'OCTPAY', 'OCTDATE', 'OCTBILL', 'NOVPAY',
       'NOVDATE', 'NOVBILL', 'DECPAY', 'DECDATE', 'DECBILL', 'REGBILL'],
      dtype='object')


In [10]:
# Since we only have first names, and a lot of the data is inconsistent
# (i.e. address in two records for the same student/parents has 'Street' and 'St')
# I think the best way to identify unique guardians is by looking at mom/dad pairings
# and dropping the duplicates (including last name)
families = STUD00.dropna(subset=['MOMNAME', 'DADNAME'], how='all'
                ).drop_duplicates(subset=['MOMNAME', 'DADNAME', 'LNAME']).sort_values(by=['LNAME','MOMNAME','DADNAME']).copy()
families.insert(0, 'FAMILY_ID', list(range(1, families.shape[0]+1)))
moms = families[~pd.isna(families['MOMNAME'])][['FAMILY_ID', 'MOMNAME', 'LNAME', 'PHONE', 'EMAIL']].rename(columns={'MOMNAME':'FNAME'})
moms.insert(1,'RELATION','MOM')
dads = families[~pd.isna(families['DADNAME'])][['FAMILY_ID', 'DADNAME', 'LNAME', 'PHONE', 'EMAIL']].rename(columns={'DADNAME':'FNAME'})
dads.insert(1,'RELATION','DAD')
guardian = pd.concat([moms, dads]).sort_values(by=['LNAME','FNAME'])
guardian.insert(0, 'GUARDIAN_ID', list(range(1,guardian.shape[0]+1)))
guardian = guardian.sort_values(by=['FAMILY_ID','GUARDIAN_ID'])
guardian.insert(len(guardian.columns),'CREA_TMS',[datetime.now()]*guardian.shape[0])
guardian.insert(len(guardian.columns),'UPDT_TMS',[datetime.now()]*guardian.shape[0])
display(guardian.tail())

,GUARDIAN_ID,FAMILY_ID,RELATION,FNAME,LNAME,PHONE,EMAIL,CREA_TMS,UPDT_TMS
1851,8904,4799,MOM,SIFISO,ZINHEMA,H/815-1504 MC/669-3909 DC/669-3855,NaN,2024-11-07 12:18:20.568845,2024-11-07 12:18:20.586574
665,8905,4800,MOM,JOSIE,ZINNINGER,MC/698-0003 DC/698-0000,NaN,2024-11-07 12:18:20.568845,2024-11-07 12:18:20.586574
665,8906,4800,DAD,SCOTT,ZINNINGER,MC/698-0003 DC/698-0000,NaN,2024-11-07 12:18:20.568845,2024-11-07 12:18:20.586574
787,8907,4801,MOM,MARIA C/813-624-5241,ZUCKER,H/813-659-2605 DC/813-763-5853,NaN,2024-11-07 12:18:20.568845,2024-11-07 12:18:20.586574
787,8908,4801,DAD,STEVEN,ZUCKER,H/813-659-2605 DC/813-763-5853,NaN,2024-11-07 12:18:20.568845,2024-11-07 12:18:20.586574


In [40]:
# Create dataframes to represent new database strucutre
student = pd.DataFrame({'STUD_ID' : STUD00['STUDENT_ID'],
                        'CLASS_ID' : [pd.NA]*STUD00.shape[0],
                        'STUDENTNO' : STUD00['STUDENTNO'],
                        'FNAME' : STUD00['FNAME'],
                        'MIDDLE' : STUD00['MIDDLE'],
                        'LNAME' :  STUD00['LNAME'],
                        'SEX' :  STUD00['SEX'],
                        'BIRTHDAY' :  STUD00['BIRTHDAY'],
                        'ENROLLDATE' :  STUD00['ENROLLDATE'],
                        'LEVEL' :  STUD00['LEVEL'],
                        'REGFEE' :  STUD00['REGFEE'],
                        'MONTHLYFEE' :  STUD00['MONTHLYFEE'],
                        'BALANCE' :  STUD00['BALANCE'],
                        'PHONE' :  STUD00['PHONE'],
                        'EMAIL' :  STUD00['EMAIL'],
                        'ADDRESS' : STUD00['ADDRESS'],
                        'CITY' : STUD00['CITY'],
                        'STATE' : STUD00['STATE'],
                        'ZIP' : STUD00['ZIP'],
                        'CREA_TMS' : [datetime.now()]*STUD00.shape[0],
                        'UPDT_TMS' : [datetime.now()]*STUD00.shape[0],})

# Insert FAMILY_ID into student
student = student.merge(STUD00.merge(families[['MOMNAME','DADNAME','LNAME','FAMILY_ID']], how='left'
                                    ).loc[:, ['STUDENTNO','FAMILY_ID']],
                        how='left', on='STUDENTNO')
family_id = student.pop('FAMILY_ID')
student.insert(1, 'FAMILY_ID', family_id)
student

,STUD_ID,FAMILY_ID,CLASS_ID,STUDENTNO,FNAME,MIDDLE,LNAME,SEX,BIRTHDAY,ENROLLDATE,...,MONTHLYFEE,BALANCE,PHONE,EMAIL,ADDRESS,CITY,STATE,ZIP,CREA_TMS,UPDT_TMS
0,1,NaN,<NA>,1001,AUDRA,A.,BILBREY,F,1982-02-13,NaN,...,0.0,0.0,H-299-9030,NaN,3257 OLD EAGLE LAKE RD.,WINTER HAVEN,FL,33880,2024-11-07 12:45:03.007686,2024-11-07 12:45:03.007698
1,2,2638.0,<NA>,1002,JACKIE,NaN,MARTIN,F,1982-12-29,NaN,...,0.0,0.0,H-537-2331 293-7140/MW-534-0860,NaN,1942 19TH ST N.W.,WINTER HAVEN,FL,33881,2024-11-07 12:45:03.007686,2024-11-07 12:45:03.007698
2,3,2766.0,<NA>,1003,KATIE,NaN,MCINTIRE,F,1979-12-05,NaN,...,0.0,0.0,644-4586,NaN,6632 W. NEWMAN CIRCLE,LAKELAND,FL,33811,2024-11-07 12:45:03.007686,2024-11-07 12:45:03.007698
3,4,626.0,<NA>,1004,CAROLYN,NaN,CABANAS,F,NaN,NaN,...,0.0,0.0,NaN,NaN,3305 HUGHES ST.,LAKELAND,FL,33801,2024-11-07 12:45:03.007686,2024-11-07 12:45:03.007698
4,5,NaN,<NA>,1005,ADAM,J,ROBINSON,M,1978-06-12,NaN,...,0.0,0.0,H-1-773-4331 MW-1-773-4556,NaN,86 JOHN'S RD.,WAUCHULA,FL,33873,2024-11-07 12:45:03.007686,2024-11-07 12:45:03.007698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5918,5919,2859.0,<NA>,7001,NA'TALYIA,NaN,MIDDLEBROOKS,NaN,2019-02-04,2024-10-21,...,100.0,0.0,M/440-1888,MCCOYKENISHA@YAHOO.COM,1024 SUSAN DR,LAKELAND,FL,33803,2024-11-07 12:45:03.007686,2024-11-07 12:45:03.007698
5919,5920,1726.0,<NA>,7002,AXEL,NaN,HANSEN,NaN,2029-03-25,2024-10-21,...,90.0,0.0,M/941-773-1700 D/272-871-2131,NaN,1334 TURKEY TRL,LAKELAND,FL,33810,2024-11-07 12:45:03.007686,2024-11-07 12:45:03.007698
5920,5921,2353.0,<NA>,7003,LAUREL,NaN,LANG,NaN,2015-10-14,2024-10-21,...,0.0,0.0,M/308-9078 D/393-2130,CASSIE3036@GMAIL.COM,4910 HERNDON WAY,AUBURNDALE,FL,33823,2024-11-07 12:45:03.007686,2024-11-07 12:45:03.007698
5921,5922,4710.0,<NA>,7005,JOANNA,NaN,WITTE,NaN,2019-04-14,2024-10-22,...,100.0,0.0,M/289-6873 D/308-4363,BOGEAMWITTE@GMAIL.COM,4665 SWINDELL RD,LAKELAND,FL,33810,2024-11-07 12:45:03.007686,2024-11-07 12:45:03.007698


In [17]:
# Load students file and list all fields
clsbymon = pd.read_csv('Data\\clsbymon.csv')
clsbymon.insert(0, 'CLASS_ID', clsbymon.index + 1)
with pd.option_context('display.max_columns', None):
    display(clsbymon.tail())

,CLASS_ID,SESSION,TEACH,TEACH2,DAYOFWEEK,TIMEOFDAY,CLASSTIME,SECONDTIME,CODE,CLASSNAME,MAX,AVAILABLE,STUDENT1,NUMB1,STUDENT2,NUMB2,STUDENT3,NUMB3,STUDENT4,NUMB4,STUDENT5,NUMB5,STUDENT6,NUMB6,STUDENT7,NUMB7,STUDENT8,NUMB8,STUDENT9,NUMB9,STUDENT10,NUMB10,STUDENT11,NUMB11,STUDENT12,NUMB12,STUDENT13,NUMB13,STUDENT14,NUMB14,STUDENT15,NUMB15,STUDENT16,NUMB16,STUDENT17,NUMB17,STUDENT18,NUMB18,STUDENT19,NUMB19,STUDENT20,NUMB20,STUDENT21,NUMB21,STUDENT22,NUMB22,STUDENT23,NUMB23,STUDENT24,NUMB24,STUDENT25,NUMB25,STUDENT26,NUMB26,STUDENT27,NUMB27,STUDENT28,NUMB28,STUDENT29,NUMB29,STUDENT30,NUMB30,STUDENT31,NUMB31,STUDENT32,NUMB32,WAIT1,W1PHONE,WAIT2,W2PHONE,WAIT3,W3PHONE,WAIT4,W4PHONE,TRIAL1,T1PHONE,T1DATE,TRIAL2,T2PHONE,T2DATE,TRIAL3,T3PHONE,T3DATE,TRIAL4,T4PHONE,T4DATE,TRIAL5,T5PHONE,T5DATE,TRIAL6,T6PHONE,T6DATE,TRIAL7,T7PHONE,T7DATE,TRIAL8,T8PHONE,T8DATE,NOTE1,NOTE2,NOTE3,NOTE4
93,94,NOV,LAUREN,NaN,3,DK,W 6:15,NaN,PS,FUNTASTIKS (4-5YRS),6,6,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,HAILEY CAMPBELL,772-812-2945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLASS FULL,NaN,NaN,ALLOW POSEY MCFADDEN,BACK IN CLASS WHEN ARM HEALS,EW 9.4.2024,NaN
94,95,NOV,KATIE,NaN,3,DH,W 6:10,NaN,GA,GIRLS INT/ADV GYMNASTICS (6 & UP),8,7,IVY MURRAY,6301,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*ESTHER BRINKLEY OWES SEP/,OCT/REG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,96,NOV,SARAH,NaN,5,AG,F 12:30,NaN,PS,FUNTASTIKS (4-5YRS),6,6,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,SAVANNAH BUPHY 4 SHYANN,709-4687,AUBREY RIMMER,NaN,LUKE LIST 9.6.24,NaN,EVERLEE ARBOGAST 5,936-581-5557,ABIGAIL KELLEY WOULD LIKE,TO,NaN,RETURN 9.20 EW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLASS FULL,NaN,NaN,NaN,NaN,NaN,NaN
96,97,NOV,SARAH,NaN,5,AH,F 1:30,NaN,PS,FUNTASTIKS (3-4YRS),6,6,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,98,NOV,SARAH,NaN,4,A,TH 9:05,NaN,GB,HOMESCHOOL (S & UP),8,8,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,SERENITY RITCHIE 8 8.16,DON 7096886,MADELYN STRINGER,704-564-6687,LEYTON&SUTTON LEDFORD,417-316-0477,KATELYN NEEL,899-9251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLASS FULL,NaN,NaN,CLOSED CLOSED CLOSED,NaN,CLOSED CLOSED HEATHER CLOSED,HEATHER CLOSED HEATHER CLOSE


In [46]:
# First 11 columns
classes = clsbymon.iloc[:,:11].copy()
classes.insert(len(classes.columns),'CREA_TMS',[datetime.now()]*classes.shape[0])
classes.insert(len(classes.columns),'UPDT_TMS',[datetime.now()]*classes.shape[0])
# 32 placeholder columns for students. For each class,
# iterate through every student column, pull student number,
# then update CLASS_ID in corresponding record in `students`
for class_id in range(clsbymon.shape[0]):
    for student_num_col in [f'NUMB{i}' for i in range(1,33)]:   
        student_num = clsbymon.loc[class_id, student_num_col]
        if student_num != 0:
            student.loc[student['STUDENTNO'] == student_num, 'CLASS_ID'] = class_id

In [108]:
# Waitlist
waitlist = clsbymon[['CLASS_ID'] + [col for i in range(1, 5) for col in (f'WAIT{i}', f'W{i}PHONE')]]
waits = []
for i in range(1,5):
    waits.append(waitlist[~pd.isna(waitlist[f'WAIT{i}'])][['CLASS_ID', f'WAIT{i}', f'W{i}PHONE']
                                                          ].rename(columns={f'WAIT{i}':'NAME',
                                                                            f'W{i}PHONE' : 'PHONE'}))

wait = pd.concat(waits).sort_values(by='CLASS_ID')
wait.insert(0, 'WAIT_ID', list(range(1, wait.shape[0]+1)))
wait.insert(len(wait.columns),'CREA_TMS',[datetime.now()]*wait.shape[0])
wait.insert(len(wait.columns),'UPDT_TMS',[datetime.now()]*wait.shape[0])
display(wait.tail())

,WAIT_ID,CLASS_ID,NAME,PHONE,CREA_TMS,UPDT_TMS
95,73,96,EVERLEE ARBOGAST 5,936-581-5557,2024-11-06 00:54:49.681609,2024-11-06 00:54:49.683713
97,74,98,SERENITY RITCHIE 8 8.16,DON 7096886,2024-11-06 00:54:49.681609,2024-11-06 00:54:49.683713
97,75,98,MADELYN STRINGER,704-564-6687,2024-11-06 00:54:49.681609,2024-11-06 00:54:49.683713
97,76,98,LEYTON&SUTTON LEDFORD,417-316-0477,2024-11-06 00:54:49.681609,2024-11-06 00:54:49.683713
97,77,98,KATELYN NEEL,899-9251,2024-11-06 00:54:49.681609,2024-11-06 00:54:49.683713


In [105]:
# Trials
trial_list = clsbymon[['CLASS_ID'] + [col for i in range(1, 9) for col in (f'TRIAL{i}', f'T{i}PHONE', f'T{i}DATE')]]
trials = []
for i in range(1,9):
    trials.append(trial_list[~pd.isna(trial_list[f'TRIAL{i}'])][['CLASS_ID', f'TRIAL{i}', f'T{i}PHONE', f'T{i}DATE']
                                                          ].rename(columns={f'TRIAL{i}' : 'NAME',
                                                                            f'T{i}PHONE' : 'PHONE',
                                                                            f'T{i}DATE' : 'DATE'}))
trial = pd.concat(trials).sort_values(by='CLASS_ID')
trial.insert(0, 'TRIAL_ID', list(range(1, trial.shape[0]+1)))
trial.insert(len(trial.columns),'CREA_TMS',[datetime.now()]*trial.shape[0])
trial.insert(len(trial.columns),'UPDT_TMS',[datetime.now()]*trial.shape[0])
display(trial.sort_values(by=['CLASS_ID','TRIAL_ID']))

,TRIAL_ID,CLASS_ID,NAME,PHONE,DATE,CREA_TMS,UPDT_TMS
0,1,1,SAVANNAH ROMAN 6 ALICIA,398-5576,2024-10-10,2024-11-06 00:46:00.162456,2024-11-06 00:46:00.164673
0,2,1,ELISE BELLOR,MU,2024-10-24,2024-11-06 00:46:00.162456,2024-11-06 00:46:00.164673
0,3,1,MADISON GENDRON,RETURNIN,2024-10-24,2024-11-06 00:46:00.162456,2024-11-06 00:46:00.164673
1,4,2,LUNA VALDEZ 2 LISA,START,2024-10-05,2024-11-06 00:46:00.162456,2024-11-06 00:46:00.164673
1,5,2,SUSIE OUT 11/2,NaN,NaN,2024-11-06 00:46:00.162456,2024-11-06 00:46:00.164673
...,...,...,...,...,...,...,...
94,159,95,*ESTHER BRINKLEY OWES SEP/,OCT/REG,NaN,2024-11-06 00:46:00.162456,2024-11-06 00:46:00.164673
95,160,96,ABIGAIL KELLEY WOULD LIKE,TO,NaN,2024-11-06 00:46:00.162456,2024-11-06 00:46:00.164673
95,161,96,RETURN 9.20 EW,NaN,NaN,2024-11-06 00:46:00.162456,2024-11-06 00:46:00.164673
95,162,96,CLASS FULL,NaN,NaN,2024-11-06 00:46:00.162456,2024-11-06 00:46:00.164673


In [109]:
# Notes
note_list = clsbymon[['CLASS_ID'] + [f'NOTE{i}' for i in range(1,5)]]
notes = []
for i in range(1,5):
    notes.append(note_list[~pd.isna(note_list[f'NOTE{i}'])][['CLASS_ID', f'NOTE{i}']
                                                          ].rename(columns={f'NOTE{i}':'NOTE_TXT'}))

note = pd.concat(notes).sort_values(by='CLASS_ID')
note.insert(0, 'NOTE_ID', list(range(1, note.shape[0]+1)))
note.insert(len(note.columns),'CREA_TMS',[datetime.now()]*note.shape[0])
note.insert(len(note.columns),'UPDT_TMS',[datetime.now()]*note.shape[0])
display(note.tail())

,NOTE_ID,CLASS_ID,NOTE_TXT,CREA_TMS,UPDT_TMS
93,51,94,EW 9.4.2024,2024-11-06 00:54:56.861218,2024-11-06 00:54:56.862026
93,52,94,BACK IN CLASS WHEN ARM HEALS,2024-11-06 00:54:56.861218,2024-11-06 00:54:56.862026
97,53,98,CLOSED CLOSED CLOSED,2024-11-06 00:54:56.861218,2024-11-06 00:54:56.862026
97,54,98,CLOSED CLOSED HEATHER CLOSED,2024-11-06 00:54:56.861218,2024-11-06 00:54:56.862026
97,55,98,HEATHER CLOSED HEATHER CLOSE,2024-11-06 00:54:56.861218,2024-11-06 00:54:56.862026


In [102]:
columns = [month.upper() + suffix for month in list(calendar.month_abbr[1:]) for suffix in ['PAY','DATE','BILL']]
payment_df = STUD00[columns]

payment_df = payment_df[((payment_df[payment_df.filter(like='PAY').columns] != 0) | (payment_df[payment_df.filter(like='BILL').columns] != '*') ).any(axis=1)]

In [120]:
import calendar
import numpy as np
# Payments
columns = [month.upper() + suffix for month in list(calendar.month_abbr[1:]) for suffix in ['PAY','DATE','BILL']]
payment_df = STUD00[['STUDENTNO'] + columns]
# Keep only the rows where there is a payment (at least one 'PAY' column which is nonzero)
payment_df = payment_df[((payment_df[payment_df.filter(like='PAY').columns] != 0) | (payment_df[payment_df.filter(like='BILL').columns] != '*') ).any(axis=1)]


payment_list = []
for row in range(payment_df.shape[0]):
    for month in range(12):
        # PAY
        amount = payment_df.iloc[row,month*3 + 1]
        # DATE
        date = payment_df.iloc[row, month*3 + 2]
        # BILL
        bill = payment_df.iloc[row, month*3 + 3]

        if (amount != 0) or (bill == '*'):

            payment_list.append(pd.DataFrame({'PAYMENT_ID' : len(payment_list) + 1,
                                 'STUDENTNO' : payment_df.iloc[row]['STUDENTNO'],
                                 'MONTH' : month+1,
                                 'PAY' : amount,
                                 'PAY_DATE' : date,
                                 'BILL' : bill,
                                 }, index=[len(payment_list)+1]))
            


payments = pd.concat(payment_list)

In [233]:
import calendar

# Payments
columns = [month.upper() + suffix for month in list(calendar.month_abbr[1:]) for suffix in ['PAY','DATE','BILL']]
payment_df = STUD00[['STUDENTNO'] + columns]
# Step 1: Reshape the DataFrame using melt
df_long = payment_df.melt(id_vars=['STUDENTNO'], var_name='variable', value_name='value')

# Step 2: Extract MONTH and TYPE from the column names
month_abbr_to_num = {calendar.month_abbr[i].upper() : i for i in range(1,13)}
df_long['MONTH'] = df_long['variable'].str[:3].map(month_abbr_to_num)
df_long['TYPE'] = df_long['variable'].str[3:]  # Remaining characters for the type

# Step 3: Group data by MONTH and row index for alignment
df_long['row'] = df_long.groupby(['STUDENTNO', 'MONTH', 'TYPE']).cumcount()

# Step 4: Pivot the table to align PAY, DATE, and BILL
df_pivot = df_long.pivot(index=['STUDENTNO','MONTH', 'row'], columns='TYPE', values='value').rename_axis(columns=None).reset_index()

# Step 5: Drop the helper index and reorder columns
df_pivot = df_pivot[['STUDENTNO', 'MONTH', 'PAY', 'DATE', 'BILL']]

payments = df_pivot[(df_pivot['PAY'] != 0) | ~pd.isna(df_pivot['BILL'])].reset_index(drop=True)
payments


,STUDENTNO,MONTH,PAY,DATE,BILL
0,1815,1,48.0,2024-01-11,NaN
1,1815,2,95.0,2024-01-29,NaN
2,3284,2,23.0,2024-02-23,NaN
3,3284,3,95.0,2024-02-23,NaN
4,3284,4,95.0,2024-04-15,NaN
...,...,...,...,...,...
5704,7001,10,25.0,2024-10-21,NaN
5705,7002,10,23.0,2024-10-21,NaN
5706,7003,10,25.0,2024-10-21,NaN
5707,7005,10,75.0,2024-10-22,NaN


#### Joining 'student' and 'guardian' (test)

In [40]:
# John Noel's student number
JN_studentno = 1897
student = pd.read_csv(data_dir + '\\rdb_format\\student.csv')
guardian = pd.read_csv(data_dir + '\\rdb_format\\guardian.csv')

# Series containing all info for a single student (capitalize all strings for visual appeal)
student_info = student.loc[student['STUDENTNO'] == JN_studentno
                            ].iloc[0
                            ].astype('string'
                            ).fillna(''
                            ).str.title()
# Dataframe containing info for student's guardians
guardian_info = guardian.loc[guardian['FAMILY_ID'] == int(float(student_info['FAMILY_ID']))]

print(student_info)
print(guardian_info)

STUD_ID                              894
FAMILY_ID                         4606.0
CLASS_ID                                
STUDENTNO                           1897
FNAME                          John Noel
MIDDLE                                  
LNAME                             Wibert
SEX                                     
BIRTHDAY                      2005-03-09
ENROLLDATE                    2008-09-11
LEVEL                                0.0
REGFEE                               0.1
MONTHLYFEE                           0.1
BALANCE                            999.0
PHONE                                   
EMAIL                                   
ADDRESS                 1337 Stratton Dr
CITY                            Lakeland
STATE                                 Fl
ZIP                              33813.0
CREA_TMS      2024-11-19 12:34:29.160329
UPDT_TMS      2024-11-19 12:34:29.160335
Name: 893, dtype: string
      GUARDIAN_ID  FAMILY_ID RELATION   FNAME   LNAME PHONE EMAIL  \
8552

#### Writing to DBF Files
So we have been able to easily read from DBF files by just converting them to CSV files and using Pandas as normal. However, we need to be able to write to DBF files too, so that the new and old programs can be run in parallel during a transition period. Eventually, there will be no need for DBF files whatsoever and the program will be fully converted over to the Python GUI using CSV files as a database. However, for the time being, the program needs to be able to write out changes to the original DBF files so that if one were to make an edit in the new program, and then switch over to the old DBASE program, those changes would be reflected in both versions.

The chunk below works. Weird syntax, but note that we can find a record in the original database by going off of the student number, and then make any changes to that record which are necessary. It is even possibl to add/remove fields, but we won't need to use that functionality; we just need to be able to 1) modify existing student records and 2) add new student records.

In [54]:
import dbf

table = dbf.Table(dbf_dir + '\\STUD00.dbf')
with table:
    studentno_idx = table.create_index(lambda rec: rec.studentno)
    # get a list of all matching records
    match = studentno_idx.search(match=2112)
    # should only be one student with that studentno
    print(len(match))
    record = match[0]
    with record:
        record.middle = 'ZZ'



1
